# Lab 1: LoRA & QLoRA - Fine-Tuning a Llama-2 Model
---
## Notebook 2: The Training Process

**Goal:** In this notebook, you will use QLoRA to fine-tune a `Llama-2-7B` model on an instruction-following dataset.

**You will learn to:**
-   Load a dataset for supervised fine-tuning and preprocess it with a tokenizer.
-   Configure `BitsAndBytesConfig` to load a model in 4-bit precision (QLoRA).
-   Load a pre-trained Llama-2 model for causal language modeling.
-   Configure `peft.LoraConfig` to apply LoRA adapters to the model.
-   Use the `transformers.Trainer` to efficiently fine-tune the model.

### Step 1: Load Dataset and Preprocess

First, we'll load our instruction-tuning dataset. We will use the `guanaco-llama2-1k` dataset, which is a small, high-quality dataset of prompts and responses formatted for Llama-2.

#### Key Hugging Face Components:

-   `datasets.load_dataset`: Fetches a dataset from the Hugging Face Hub.
-   `transformers.AutoTokenizer`: Loads the appropriate tokenizer for our model.
-   `dataset.map()`: A powerful method to apply a processing function to every example in the dataset. We use `batched=True` for efficient processing.

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "NousResearch/Llama-2-7b-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Important for Causal LM

# --- Load Dataset ---
dataset = load_dataset(dataset_name, split="train")

# --- Preprocess and Tokenize ---
def preprocess_function(examples):
    # The 'text' field contains the full formatted prompt.
    return tokenizer(examples["text"], truncation=True, max_length=1024)

# Split the dataset
dataset = dataset.train_test_split(test_size=0.1)

# Tokenize both splits
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"]) # Remove original text column

print("✅ Dataset loaded and tokenized.")
print(f"Train samples: {len(tokenized_datasets['train'])}")
print(f"Test samples: {len(tokenized_datasets['test'])}")

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Dataset loaded and tokenized.
Train samples: 900
Test samples: 100


### Step 2: Load the Base Model

Now, we load the base model. The key to QLoRA is loading the base model in a quantized format.

#### Key Hugging Face `transformers` Components:

-   `transformers.BitsAndBytesConfig`: This configuration class is used to specify all the parameters for quantization.
    -   `load_in_4bit=True`: This is the master switch to enable 4-bit loading.
    -   `bnb_4bit_quant_type="nf4"`: We use the "nf4" (Normalized Float 4) quantization type, which is recommended for QLoRA.
    -   `bnb_4bit_compute_dtype=torch.bfloat16`: This sets the compute data type during the forward and backward passes. `bfloat16` is a good choice for modern GPUs.
-   `transformers.AutoModelForCausalLM`: We use this to load our Llama-2 model, passing the `quantization_config` to it.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# --- Quantization Configuration ---
# Load model in 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# --- Load Base Model ---
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=quantization_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

print("✅ Base model loaded successfully!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Step 3: Configure LoRA

Now we configure LoRA using the `peft` library.

#### Key Hugging Face `peft` Components:

-   `peft.LoraConfig`: The main configuration class for LoRA.
    -   `r`: The rank of the update matrices. A lower rank means fewer trainable parameters. A common range is 8-64.
    -   `lora_alpha`: The scaling factor for the LoRA matrices. It's often set to twice the rank (`2*r`).
    -   `target_modules`: A list of the names of the modules (e.g., attention layers) to apply LoRA to. For Llama models, this is typically `["q_proj", "k_proj", "v_proj", "o_proj"]`.
    -   `lora_dropout`: Dropout probability for the LoRA layers to reduce overfitting.
    -   `bias="none"`: Specifies which biases to train. "none" is common.
    -   `task_type="CAUSAL_LM"`: Specifies the task type.
-   `peft.get_peft_model`: This function takes the base model and the LoRA config and returns a PEFT model ready for training.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# --- LoRA Configuration ---
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# --- Create PEFT Model ---
peft_model = get_peft_model(model, lora_config)

# --- Print Trainable Parameters ---
peft_model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.2484


### Step 4: Set Up Training

The final step is to configure and run the training process using the `transformers.Trainer`.

#### Key Hugging Face Components:

-   `transformers.TrainingArguments`: This class holds all the hyperparameters for the training run, such as learning rate, number of epochs, batch size, and logging settings.
    -   `metric_for_best_model="perplexity"`: We specify perplexity as the metric to determine the best model.
    -   `greater_is_better=False`: Lower perplexity indicates better performance.
-   `transformers.Trainer`: The standard trainer class from the `transformers` library. It requires a model, training arguments, datasets, a tokenizer, and a data collator.
-   `transformers.DataCollatorForLanguageModeling`: This data collator will be used to form batches of tokenized data. It also handles the creation of the `labels` for causal language modeling, where the model predicts the next token.
-   `compute_metrics`: A custom function to calculate **perplexity**, which is the standard evaluation metric for language modeling tasks. Perplexity measures how well the model predicts the next token - lower values indicate better performance.

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
import math
import numpy as np

# --- Custom Evaluation Metrics ---
def compute_metrics(eval_pred):
    """
    Compute perplexity from the model's predictions.
    Perplexity is the standard metric for language modeling tasks.
    """
    predictions, labels = eval_pred
    
    # Convert to tensors
    predictions = torch.from_numpy(predictions).float()
    labels = torch.from_numpy(labels).long()
    
    # Shift for causal language modeling
    shift_logits = predictions[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    
    # Calculate cross entropy loss
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(
        shift_logits.view(-1, shift_logits.size(-1)), 
        shift_labels.view(-1)
    )

    # Calculate perplexity
    try:
        perplexity = math.exp(loss.item())
    except OverflowError:
        perplexity = float('inf')
    
    return {
        "perplexity": perplexity,
        "eval_loss": loss.item()
    }

# --- Data Collator ---
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir="./lora-llama2-7b-guanaco",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_steps=2,
    eval_strategy="steps",  # 改為 steps 以便更頻繁顯示指標
    eval_steps=50,          # 每50步評估一次
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="perplexity",
    greater_is_better=False,  # Lower perplexity is better
    logging_first_step=True,  # 記錄第一步
    report_to=None,          # 避免外部報告干擾
)

# --- Create Trainer ---
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- Start Training ---
print("🚀 Starting training with LoRA...")
trainer.train()
print("✅ Training complete!")

# --- Final Evaluation ---
print("\n📊 Final Evaluation Results:")
final_metrics = trainer.evaluate()
print(f"Final Perplexity: {final_metrics['eval_perplexity']:.4f}")
print(f"Final Loss: {final_metrics['eval_loss']:.4f}")